### data preprocessing

In [4]:
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

import numpy as np # Assuming y_train might need checking for type
import logging # Import the logging library
import sys # To log to stdout as well
import time # To timestamp log file
# Import libraries


import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
from scipy.stats import chi2_contingency

In [2]:
bookings = pd.read_csv('https://raw.githubusercontent.com/datacamp/Machine-Learning-With-XGboost-live-training/master/data/hotel_bookings_clean.csv')

# List out our columns
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119210 entries, 0 to 119209
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   is_canceled                     119210 non-null  int64  
 1   lead_time                       119210 non-null  int64  
 2   arrival_date_week_number        119210 non-null  int64  
 3   arrival_date_day_of_month       119210 non-null  int64  
 4   arrival_date_month              119210 non-null  int64  
 5   stays_in_weekend_nights         119210 non-null  int64  
 6   stays_in_week_nights            119210 non-null  int64  
 7   adults                          119210 non-null  int64  
 8   children                        119206 non-null  float64
 9   babies                          119210 non-null  int64  
 10  is_repeated_guest               119210 non-null  int64  
 11  previous_cancellations          119210 non-null  int64  
 12  previous_booking

In [3]:
X, y = bookings.iloc[:,1:], bookings.iloc[:,0]

In [ ]:
# def modelfit(alg, X, y, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(X, label=y)
#         cvresult = xgb.cv(xgb_param, 
#                           xgtrain, 
#                           num_boost_round=alg.get_params()['n_estimators'], 
#                           nfold=cv_folds,
#                           metrics='auc', 
#                           early_stopping_rounds=early_stopping_rounds, 
#                           show_progress=True)
        
#         alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    # alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    # #Predict training set:
    # dtrain_predictions = alg.predict(dtrain[predictors])
    # dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    # #Print model report:
    # print("\nModel Report")
    # print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    # print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    # feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    # feat_imp.plot(kind='bar', title='Feature Importances')
    # plt.ylabel('Feature Importance Score')

In [ ]:
#Choose all predictors except target & IDcols

# xgb1 = xgb.XGBClassifier(
#  learning_rate =0.1,
#  n_estimators=4000,
#  max_depth=5,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  n_jobs=-1,
#  scale_pos_weight=1,
#  seed=27)

# modelfit(xgb1, X_train_filtered, y_train)

### step1: tune estimater

In [6]:
genes_dmatrix = xgb.DMatrix(data=X, label =y)
params = {
    'objective': 'binary:logistic',
    'eta': 0.1, # Corresponds to learning_rate
    'max_depth': 5,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'scale_pos_weight': 3,
    'seed': 27, # Seed for internal XGBoost randomness (like feature subsampling)
    'nthread': -1
}

cv_results = xgb.cv(
    params = params,
    dtrain = genes_dmatrix,
    num_boost_round = 4000,
    nfold = 5,
    metrics = {'logloss', 'auc'},
    early_stopping_rounds = 50,
    seed = 27,
    verbose_eval = 50
    )



[0]	train-logloss:0.76233+0.00034	train-auc:0.82925+0.00103	test-logloss:0.76236+0.00126	test-auc:0.82912+0.00334
[50]	train-logloss:0.48158+0.00184	train-auc:0.87839+0.00117	test-logloss:0.48367+0.00181	test-auc:0.87630+0.00239
[100]	train-logloss:0.45651+0.00081	train-auc:0.89370+0.00047	test-logloss:0.46194+0.00312	test-auc:0.88892+0.00249
[150]	train-logloss:0.43994+0.00111	train-auc:0.90306+0.00053	test-logloss:0.44876+0.00279	test-auc:0.89571+0.00233
[200]	train-logloss:0.42775+0.00062	train-auc:0.90930+0.00032	test-logloss:0.44009+0.00321	test-auc:0.89949+0.00234
[250]	train-logloss:0.41721+0.00118	train-auc:0.91463+0.00057	test-logloss:0.43306+0.00299	test-auc:0.90258+0.00207
[300]	train-logloss:0.40877+0.00124	train-auc:0.91886+0.00056	test-logloss:0.42787+0.00319	test-auc:0.90475+0.00217
[350]	train-logloss:0.40123+0.00142	train-auc:0.92245+0.00062	test-logloss:0.42344+0.00325	test-auc:0.90646+0.00218
[400]	train-logloss:0.39444+0.00134	train-auc:0.92567+0.00055	test-logloss:

In [16]:
# cv_results
cv_results.tail()
# optimal_rounds = cv_results.shape[0]
# print(optimal_rounds)


,train-logloss-mean,train-logloss-std,train-auc-mean,train-auc-std,test-logloss-mean,test-logloss-std,test-auc-mean,test-auc-std
7,0.680279,0.002629,0.623371,0.007216,0.690569,0.002600,0.506240,0.009338
8,0.679318,0.002594,0.630752,0.008529,0.690579,0.002524,0.508000,0.008719
9,0.678173,0.002669,0.634738,0.008981,0.689942,0.002353,0.510875,0.009142
10,0.677020,0.002146,0.637230,0.008550,0.689755,0.002516,0.511950,0.008992
11,0.675554,0.002405,0.642632,0.009216,0.689426,0.002295,0.513750,0.010016


In [38]:
#base model
xgb1_1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=120,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 n_jobs=-1,
 scale_pos_weight=scale_pos_weight_val,
 seed=27)

xgb1_1.fit(X_train_filtered, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=120,
              n_jobs=-1, num_parallel_tree=None, ...)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

y_pred_proba = xgb1_1.predict_proba(X_test_filtered)[:,1]
y_pred = xgb1_1.predict(X_test_filtered)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# Precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)
# Recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)
# F1 Score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


print(f"Final Model ROC AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nFinal Classification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
 [[102 142]
 [370 357]]
Accuracy: 0.4727085478887745
Precision: 0.7154308617234469
Recall: 0.49105914718019256
F1 Score: 0.5823817292006526
Final Model ROC AUC: 0.4722

Final Classification Report:
              precision    recall  f1-score   support

           0       0.22      0.42      0.28       244
           1       0.72      0.49      0.58       727

    accuracy                           0.47       971
   macro avg       0.47      0.45      0.43       971
weighted avg       0.59      0.47      0.51       971



### Step2: Tune max_depth and min_child_weigh

In [24]:
param_test1 = {
#  'max_depth':range(3,10,2),
#  'min_child_weight':range(1,6,2)

 'max_depth':[4,5,6],#6
 'min_child_weight':[6,8,10,12] #6
}
xgb_2_1=xgb.XGBClassifier( learning_rate =0.1, 
                        n_estimators=12, 
                        max_depth=5,
                        min_child_weight=1, 
                        gamma=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8,
                        objective= 'binary:logistic', 
                        nthread=4, 
                        scale_pos_weight=scale_pos_weight_val, 
                        seed=27)
gsearch1 = GridSearchCV(estimator = xgb_2_1, 
                        param_grid = param_test1, 
                        scoring='roc_auc',
                        n_jobs=-1, 
                        cv=5)

gsearch1.fit(X_train_filtered,y_train)

results_df = pd.DataFrame(gsearch1.cv_results_)
print(results_df)
print(results_df.sort_values(by='rank_test_score'))

print(f"best_params:{gsearch1.best_params_}") 
print(f"Best_score:{gsearch1.best_score_}")

/hdd1/home/f24_yqzhou/miniconda3/envs/6010/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/hdd1/home/f24_yqzhou/miniconda3/envs/6010/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        2.970508      0.425429         0.396609        0.026358   
1        2.834768      0.456463         0.405674        0.006758   
2        3.068122      0.192124         0.368063        0.044089   
3        2.747983      0.487982         0.389913        0.033693   
4        2.421840      0.265088         0.415112        0.081698   
5        3.001717      0.459126         0.397114        0.011725   
6        2.873758      0.426151         0.396362        0.016078   
7        2.664079      0.475909         0.398980        0.008429   
8        2.078604      0.128365         0.396349        0.008481   
9        2.305181      0.555536         0.438159        0.085946   
10       2.229320      0.502983         0.397833        0.008538   
11       2.069347      0.133278         0.395726        0.005745   

    param_max_depth  param_min_child_weight  \
0                 4                       6   
1                 4  

### Step3: tune gamma

In [ ]:
param_test1 = {
'gamma':[i/10.0 for i in range(0,5)] #0.2
}
xgb_2_1=xgb.XGBClassifier( learning_rate =0.1, 
                        n_estimators=12, 
                        max_depth=6,
                        min_child_weight=6, 
                        gamma=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8,
                        objective= 'binary:logistic', 
                        nthread=4, 
                        scale_pos_weight=scale_pos_weight_val, 
                        seed=27)
gsearch1 = GridSearchCV(estimator = xgb_2_1, 
                        param_grid = param_test1, 
                        scoring='roc_auc',
                        n_jobs=-1, 
                        cv=5)

gsearch1.fit(X_train_filtered,y_train)

results_df = pd.DataFrame(gsearch1.cv_results_)
print(results_df)
print(results_df.sort_values(by='rank_test_score'))

print(f"best_params:{gsearch1.best_params_}") 
print(f"Best_score:{gsearch1.best_score_}")

/hdd1/home/f24_yqzhou/miniconda3/envs/6010/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/hdd1/home/f24_yqzhou/miniconda3/envs/6010/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  param_gamma  \
0       2.032600      0.024619         0.403394        0.008197          0.0   
1       2.050518      0.020248         0.395679        0.009032          0.1   
2       1.976160      0.143037         0.375062        0.046722          0.2   
3       2.053400      0.007613         0.401876        0.005214          0.3   
4       2.071623      0.032929         0.411520        0.011318          0.4   

           params  split0_test_score  split1_test_score  split2_test_score  \
0  {'gamma': 0.0}           0.524489            0.53983           0.546758   
1  {'gamma': 0.1}           0.524489            0.53983           0.545699   
2  {'gamma': 0.2}           0.524489            0.53983           0.545699   
3  {'gamma': 0.3}           0.524489            0.53983           0.545699   
4  {'gamma': 0.4}           0.524489            0.53983           0.545699   

   split3_test_score  split4_test_score  mean_test

### Step 4: Tune subsample and colsample_bytree

In [ ]:
param_test1 = {
'subsample':[i/10.0 for i in range(1,10)], #0.2
 'colsample_bytree':[i/10.0 for i in range(1,10)] #0.7
}
xgb_2_1=xgb.XGBClassifier( learning_rate =0.1, 
                        n_estimators=12, 
                        max_depth=6,
                        min_child_weight=6, 
                        gamma=0.2, 
                        subsample=0.8, 
                        colsample_bytree=0.8,
                        objective= 'binary:logistic', 
                        nthread=4, 
                        scale_pos_weight=scale_pos_weight_val, 
                        seed=27)
gsearch1 = GridSearchCV(estimator = xgb_2_1, 
                        param_grid = param_test1, 
                        scoring='roc_auc',
                        n_jobs=-1, 
                        cv=5)

gsearch1.fit(X_train_filtered,y_train)

results_df = pd.DataFrame(gsearch1.cv_results_)
print(results_df)
print(results_df.sort_values(by='rank_test_score'))

print(f"best_params:{gsearch1.best_params_}") 
print(f"Best_score:{gsearch1.best_score_}")

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        4.731436      1.491592         0.547544        0.019983   
1        4.848363      2.028891         0.651835        0.094777   
2        5.476722      1.724943         0.590022        0.022809   
3        6.360034      1.636496         0.590580        0.048307   
4        6.288605      1.445792         0.642595        0.028535   
..            ...           ...              ...             ...   
76       2.368969      0.163884         0.275377        0.016652   
77       2.249490      0.131008         0.299529        0.020305   
78       2.252794      0.090927         0.279917        0.017869   
79       2.099086      0.229107         0.296693        0.009932   
80       2.072867      0.108699         0.270310        0.007329   

    param_colsample_bytree  param_subsample  \
0                      0.1              0.1   
1                      0.1              0.2   
2                      0.1              0.

### Step 5: Tuning Regularization Parameters

In [33]:
param_test1 = {
'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05] #0.001
}
xgb_2_1=xgb.XGBClassifier( learning_rate =0.1, 
                        n_estimators=12, 
                        max_depth=6,
                        min_child_weight=6, 
                        gamma=0.2, 
                        subsample=0.7, 
                        colsample_bytree=0.2,
                        objective= 'binary:logistic', 
                        nthread=4, 
                        scale_pos_weight=scale_pos_weight_val, 
                        seed=27)
gsearch1 = GridSearchCV(estimator = xgb_2_1, 
                        param_grid = param_test1, 
                        scoring='roc_auc',
                        n_jobs=-1, 
                        cv=5)

gsearch1.fit(X_train_filtered,y_train)

results_df = pd.DataFrame(gsearch1.cv_results_)
print(results_df)
print(results_df.sort_values(by='rank_test_score'))

print(f"best_params:{gsearch1.best_params_}") 
print(f"Best_score:{gsearch1.best_score_}")

/hdd1/home/f24_yqzhou/miniconda3/envs/6010/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       1.781503      0.094548         0.363194        0.016390   
1       1.764673      0.093487         0.356649        0.042511   
2       1.735071      0.063197         0.429946        0.012459   
3       1.713412      0.077029         0.390649        0.025509   
4       1.725825      0.051106         0.362806        0.063024   

   param_reg_alpha                params  split0_test_score  \
0            0.000      {'reg_alpha': 0}           0.532521   
1            0.001  {'reg_alpha': 0.001}           0.532592   
2            0.005  {'reg_alpha': 0.005}           0.532601   
3            0.010   {'reg_alpha': 0.01}           0.532610   
4            0.050   {'reg_alpha': 0.05}           0.531665   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0           0.543890           0.578371           0.528900           0.537088   
1           0.543899           0.578371           0.5289

### retune estimator

In [40]:



genes_dmatrix = xgb.DMatrix(data=X_train_filtered, label =y_train)
params = {
    'objective': 'binary:logistic',
    'eta': 0.1, # Corresponds to learning_rate
    'max_depth': 6,
    'min_child_weight': 6,
    'gamma': 0.2,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'scale_pos_weight': scale_pos_weight_val,
    'seed': 27, # Seed for internal XGBoost randomness (like feature subsampling)
    'nthread': -1,
    'reg_alpha': 0.001
}

cv_results = xgb.cv(
    params = params,
    dtrain = genes_dmatrix,
    num_boost_round = 4000,
    nfold = 5,
    metrics = {'logloss', 'auc'},
    early_stopping_rounds = 50,
    seed = 27,
    verbose_eval = 5
    )


[0]	train-logloss:0.69291+0.00309	train-auc:0.53152+0.00246	test-logloss:0.69356+0.00294	test-auc:0.51079+0.01433
[5]	train-logloss:0.68966+0.00328	train-auc:0.56154+0.00330	test-logloss:0.69170+0.00374	test-auc:0.52728+0.01091
[10]	train-logloss:0.68778+0.00338	train-auc:0.57083+0.00462	test-logloss:0.69126+0.00363	test-auc:0.52641+0.00754
[15]	train-logloss:0.68605+0.00365	train-auc:0.57148+0.00303	test-logloss:0.69082+0.00352	test-auc:0.52526+0.00620
[20]	train-logloss:0.68459+0.00364	train-auc:0.57692+0.00187	test-logloss:0.69042+0.00332	test-auc:0.52360+0.00770
[25]	train-logloss:0.68250+0.00235	train-auc:0.58267+0.00263	test-logloss:0.68929+0.00317	test-auc:0.52877+0.00913
[30]	train-logloss:0.68259+0.00228	train-auc:0.58376+0.00296	test-logloss:0.69049+0.00295	test-auc:0.52799+0.00831
[35]	train-logloss:0.68265+0.00249	train-auc:0.58380+0.00308	test-logloss:0.69141+0.00350	test-auc:0.52632+0.00950
[40]	train-logloss:0.68131+0.00297	train-auc:0.58732+0.00240	test-logloss:0.69119+

In [41]:
# cv_results
cv_results.tail()
# optimal_rounds = cv_results.shape[0]
# print(optimal_rounds)


,train-logloss-mean,train-logloss-std,train-auc-mean,train-auc-std,test-logloss-mean,test-logloss-std,test-auc-mean,test-auc-std
19,0.684830,0.003404,0.576018,0.003134,0.690420,0.002878,0.522379,0.005253
20,0.684591,0.003635,0.576925,0.001873,0.690424,0.003317,0.523600,0.007698
21,0.684440,0.002700,0.578172,0.002190,0.690416,0.002624,0.526965,0.005657
22,0.684126,0.002500,0.579772,0.002357,0.690231,0.002767,0.530925,0.008930
23,0.683574,0.002519,0.580710,0.001798,0.689773,0.002944,0.531280,0.009691


In [42]:
optimal_rounds = cv_results.shape[0]
print(optimal_rounds)

24


In [44]:
xgb3 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=24,
 max_depth=6,
 min_child_weight=6,
 gamma=0.2,
 subsample=0.7,
 colsample_bytree=0.2,
 objective= 'binary:logistic',
 n_jobs=-1,
 scale_pos_weight=scale_pos_weight_val,
 seed=27,
 reg_alpha = 0.001
 )



xgb3.fit(X_train_filtered, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.2, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=6, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=24,
              n_jobs=-1, num_parallel_tree=None, ...)

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

y_pred_proba = xgb3.predict_proba(X_test_filtered)[:,1]
y_pred = xgb3.predict(X_test_filtered)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# Precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)
# Recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)
# F1 Score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


print(f"Final Model ROC AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nFinal Classification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
 [[183  61]
 [516 211]]
Accuracy: 0.4057672502574665
Precision: 0.7757352941176471
Recall: 0.2902338376891334
F1 Score: 0.42242242242242245
Final Model ROC AUC: 0.5242

Final Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.75      0.39       244
           1       0.78      0.29      0.42       727

    accuracy                           0.41       971
   macro avg       0.52      0.52      0.41       971
weighted avg       0.65      0.41      0.41       971

